# 🚀 Showcase: 2D to 360° Video Generation (Robust Version)

This notebook provides a comprehensive and robust showcase for the 'Beyond the Frame' project. It has been redesigned to be more resilient to common errors.

### **Instructions**

1.  **Read Each Step Carefully:** Follow the instructions in the markdown cells. Do not skip steps.
2.  **GPU Runtime:** Ensure your runtime is set to use a GPU (`Runtime` > `Change runtime type` > `T4 GPU`).
3.  **Run Cells in Order:** Execute each cell from top to bottom.

### **Step 0: Factory Reset (Optional but Recommended)**

If you have run this notebook before, running the cell below will delete the old `argus-code` directory. This prevents pathing conflicts and ensures a clean setup. It's highly recommended to run this if you are re-running the notebook.

In [ ]:
print('Cleaning up previous runs...')
!rm -rf argus-code
print('Cleanup complete.')

### **Step 1: Install Conda & Restart Kernel**

⚠️ **CRITICAL STEP** ⚠️

Running the cell below will install `conda` and **automatically restart the Colab kernel.** This is expected and necessary.

**After the kernel restarts, you must continue running the cells from Step 2.**

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

--- 
### **Step 2: Full Setup (Run This After Kernel Restart)**

**IMPORTANT:** Run this single cell *after* the kernel has restarted from Step 1. 

This cell will:
1.  Clone the project repository.
2.  Install all dependencies in the correct order.
3.  Download all required AI models.
4.  Verify that all downloads and files are in the correct place.

In [ ]:
import os

print('Cloning repository...')
!git clone --recurse-submodules https://github.com/Red-Fairy/argus-code
os.chdir('argus-code')

print('✅ Repository cloned.')

print('\nInstalling dependencies...')
# Install core conda packages first
!conda install -y -c pytorch -c nvidia faiss-gpu=1.8.0 cython

# Install pip packages
!pip install -q -r requirements.txt
!pip install -q yt-dlp gdown

!git clone https://github.com/jenicek/asmk
%cd asmk/cython/
!cythonize *.pyx
%cd ..
!pip install .
%cd ..
print('✅ Dependencies installed.')

print('\nDownloading pre-trained models...')
!mkdir -p checkpoints
!gdown --id 1mZpViQY2yvwav-CcxdsQouYu8t52b25G -O checkpoints/pretrained-weights.zip

!mkdir -p mega-sam/Depth-Anything/checkpoints
!wget -q https://huggingface.co/spaces/LiheYoung/Depth-Anything/resolve/main/checkpoints/depth_anything_vitl14.pth -O mega-sam/Depth-Anything/checkpoints/depth_anything_vitl14.pth

!mkdir -p mega-sam/cvd_opt
!gdown --id 1sWDsfuZ3Up38EUQt7-JDTT1HcGHuJgvT -O mega-sam/cvd_opt/raft-things.pth
print('✅ Models downloaded.')

print('\nVerifying files...')
!unzip -q checkpoints/pretrained-weights.zip -d checkpoints/

files_to_check = [
    'checkpoints/pretrained-weights/unet/diffusion_pytorch_model.safetensors',
    'mega-sam/Depth-Anything/checkpoints/depth_anything_vitl14.pth',
    'mega-sam/cvd_opt/raft-things.pth'
]

all_files_exist = True
for f in files_to_check:
    if not os.path.exists(f):
        print(f'❌ VERIFICATION FAILED: Could not find {f}. The download or unzip may have failed.')
        all_files_exist = False

if all_files_exist:
    print('✅ All files verified successfully!')
    print('\n--- SETUP COMPLETE ---')
else:
    print('\n--- SETUP FAILED: Please review the errors above. ---')

### **Step 3: Choose Your Demo**

**Important:** Only run the cells for the option you want to use.

---

### ⚠️ **A Note on Resources** ⚠️

Video generation is **extremely resource-intensive**. The free T4 GPU provided by Colab has limited memory.

*   **Out of Memory Errors:** If you try to process a long or high-resolution video, you will likely encounter an 'Out of Memory' (OOM) error. 
*   **Recommendation:** For your first run, use a very short video (e.g., 5-10 seconds) to ensure the entire pipeline works. Once confirmed, you can experiment with longer videos, but be aware of the hardware limitations.

---

#### Option 1: Interactive Demo (Gradio Web UI)

Run the cell below to launch a Gradio web interface. A public URL will be generated. Click it to open the demo in a new tab, where you can upload your own video for processing.

In [ ]:
!python gradio_demo.py --share

---

#### Option 2: Automated Processing from a URL

Run the two cells below to automatically download a video from a URL and process it using the command-line script.

In [ ]:
# --- You can change this URL to any video you want to process ---
video_url = 'https://www.youtube.com/watch?v=YE7VzlLtp-4' # Big Buck Bunny (placeholder)

!mkdir -p input_videos
!yt-dlp -o 'input_videos/video.mp4' -f 'best[height<=720]' {video_url}

video_path = 'input_videos/video.mp4'
if os.path.exists(video_path):
    print(f'✅ Video downloaded successfully to: {video_path}')
else:
    print(f'❌ FAILED to download video. The URL may be invalid.')

In [ ]:
if os.path.exists(video_path):
    print('Starting automated processing...')
    unet_path = './checkpoints/pretrained-weights'
    save_folder = './outputs'
    
    !python inference.py \
        --unet_path {unet_path} \
        --video_path {video_path} \
        --val_save_folder {save_folder} \
        --guidance_scale 3 \
        --num_inference_steps 25
    
    print(f'\n--- Processing Complete! ---\n')
    print(f'Check the `{save_folder}` directory for your generated 360° video.')
else:
    print('Skipping processing because video download failed.')